In [1]:
#project 1

In [2]:
import sys
import argparse
import numpy as np
import time
import zipfile


def parse_reads_file(reads_fn):
    """
    :param reads_fn: the file containing all of the reads
    :return: outputs a list of all paired-end reads
    """
    try:
        with open(reads_fn, 'r') as rFile:
            print("Parsing Reads")
            first_line = True
            count = 0
            all_reads = []
            for line in rFile:
                count += 1
                if count % 1000 == 0:
                    print(count, " reads done")
                if first_line:
                    first_line = False
                    continue
                ends = line.strip().split(',')
                all_reads.append(ends)
        return all_reads
    except IOError:
        print("Could not read file: ", reads_fn)
        return None


def parse_ref_file(ref_fn):
    """
    :param ref_fn: the file containing the reference genome
    :return: a string containing the reference genome
    """
    try:
        with open(ref_fn, 'r') as gFile:
            print("Parsing Ref")
            first_line = True
            ref_genome = ''
            for line in gFile:
                if first_line:
                    first_line = False
                    continue
                ref_genome += line.strip()
        return ref_genome
    except IOError:
        print("Could not read file: ", ref_fn)
        return None

input_reads = parse_reads_file('reads_hw1_W_2_chr_1.txt')
reference = parse_ref_file('ref_hw1_W_2_chr_1.txt')


Parsing Reads
1000  reads done
2000  reads done
3000  reads done
Parsing Ref


In [8]:
len(reference)

10000

In [14]:
len(reference[(10000-25):10000])

25

In [40]:
hash_table={}
for i in range(len(reference)-25):
    seq=reference[i:i+25]
    if seq not in hash_table:
        hash_table[seq]=[i]
    else:
        hash_table[seq]=hash_table[seq]+[i]

In [41]:
seq_table=[]
for line in input_reads:
    st1=line[0]
    seq_table=seq_table+[[st1[0:25],st1[25:50]]]
    st2=line[1]
    seq_table=seq_table+[[st2[0:25],st2[25:50]]]

In [66]:
cand_snp=[]
for seqs in seq_table:
    seq1=seqs[0]
    seq2=seqs[1]
    seq=seq1+seq2
    if seq1 in hash_table:
        inds=hash_table[seq1]
        for ind in inds:
            SNP_ind_list=[]
            if ind<=9950:
                for i in range(25,50):
                    compare_seq1=seq[i]
                    compare_seq2=reference[ind+i]
                    if compare_seq1!=compare_seq2:
                        SNP_ind_list=SNP_ind_list+[[compare_seq2,compare_seq1,ind+i]]
                if len(SNP_ind_list)<=3 and len(SNP_ind_list)>0:
                    cand_snp=cand_snp+SNP_ind_list
    if seq2 in hash_table:
        inds=hash_table[seq2]
        for ind in inds:
            SNP_ind_list=[]
            if ind>=25:
                for i in range(25):
                    compare_seq1=seq1[-i]
                    compare_seq2=reference[ind-i]
                    if compare_seq1!=compare_seq2:
                        SNP_ind_list=SNP_ind_list+[[compare_seq2,compare_seq1,ind-i]]
                if len(SNP_ind_list)<=3 and len(SNP_ind_list)>0:
                    cand_snp=cand_snp+SNP_ind_list

In [67]:
pos_count={}
for cand in cand_snp:
    pos=cand[2]
    if pos not in pos_count:
        pos_count[pos]=1
    else:
        pos_count[pos]+=1

In [68]:
ret=[]
for cand in cand_snp:
    pos=cand[2]
    if pos_count[pos]>=5:
        if cand not in ret:
            ret=ret+[cand]
len(ret)

30

In [ ]:
import sys
import argparse
import numpy as np
import time
import zipfile


def parse_reads_file(reads_fn):
    """
    :param reads_fn: the file containing all of the reads
    :return: outputs a list of all paired-end reads
    """
    try:
        with open(reads_fn, 'r') as rFile:
            print("Parsing Reads")
            first_line = True
            count = 0
            all_reads = []
            for line in rFile:
                count += 1
                if count % 1000 == 0:
                    print(count, " reads done")
                if first_line:
                    first_line = False
                    continue
                ends = line.strip().split(',')
                all_reads.append(ends)
        return all_reads
    except IOError:
        print("Could not read file: ", reads_fn)
        return None


def parse_ref_file(ref_fn):
    """
    :param ref_fn: the file containing the reference genome
    :return: a string containing the reference genome
    """
    try:
        with open(ref_fn, 'r') as gFile:
            print("Parsing Ref")
            first_line = True
            ref_genome = ''
            for line in gFile:
                if first_line:
                    first_line = False
                    continue
                ref_genome += line.strip()
        return ref_genome
    except IOError:
        print("Could not read file: ", ref_fn)
        return None


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='basic_aligner.py takes in data for homework assignment 1 consisting '
                                     'of a genome and a set of reads and aligns the reads to the reference genome, '
                                     'then calls SNPs based on this alignment')
    parser.add_argument('-g', '--referenceGenome', required=True, dest='reference_file',
                        help='File containing a reference genome.')
    parser.add_argument('-r', '--reads', required=True, dest='reads_file',
                        help='File containg sequencing reads.')
    parser.add_argument('-o', '--outputFile', required=True, dest='output_file',
                        help='Output file name.')
    parser.add_argument('-t', '--outputHeader', required=True, dest='output_header',
                        help='String that needs to be outputted on the first line of the output file so that the '
                             'online submission system recognizes which leaderboard this file should be submitted to.'
                             'This HAS to be practice_W_1_chr_1 for the practice data and hw1_W_2_chr_1 for the '
                             'for-credit assignment!')
    args = parser.parse_args()
    reference_fn = args.reference_file
    reads_fn = args.reads_file

    input_reads = parse_reads_file(reads_fn)
    if input_reads is None:
        sys.exit(1)
    reference = parse_ref_file(reference_fn)
    if reference is None:
        sys.exit(1)
        
    hash_table={}
    for i in range(len(reference)-25):
        seq=reference[i:i+25]
        if seq not in hash_table:
            hash_table[seq]=[i]
        else:
            hash_table[seq]=hash_table[seq]+[i]  #build the hash table for checking
    
    seq_table=[]
    for line in input_reads:
        st1=line[0]
        seq_table=seq_table+[[st1[0:25],st1[25:50]]]
        st2=line[1]
        seq_table=seq_table+[[st2[0:25],st2[25:50]]]  #build the sequences for checking
    
    cand_snp=[]
    for seqs in seq_table:
        seq1=seqs[0]
        seq2=seqs[1]
        seq=seq1+seq2
        if seq1 in hash_table:
            inds=hash_table[seq1]
            for ind in inds:
                SNP_ind_list=[]
                if ind<=9950:
                    for i in range(25,50):
                        compare_seq1=seq[i]
                        compare_seq2=reference[ind+i]
                        if compare_seq1!=compare_seq2:
                            SNP_ind_list=SNP_ind_list+[[compare_seq2,compare_seq1,ind+i]]
                    if len(SNP_ind_list)<=3 and len(SNP_ind_list)>0:
                        cand_snp=cand_snp+SNP_ind_list
        if seq2 in hash_table:
            inds=hash_table[seq2]
            for ind in inds:
                SNP_ind_list=[]
                if ind>=25:
                    for i in range(25):
                        compare_seq1=seq1[-i]
                        compare_seq2=reference[ind-i]
                        if compare_seq1!=compare_seq2:
                            SNP_ind_list=SNP_ind_list+[[compare_seq2,compare_seq1,ind-i]] #find the candidate SNPs
                    if len(SNP_ind_list)<=3 and len(SNP_ind_list)>0:
                        cand_snp=cand_snp+SNP_ind_list
    pos_count={}
    for cand in cand_snp:
        pos=cand[2]
        if pos not in pos_count:   #find the appearance for each position
            pos_count[pos]=1
        else:
            pos_count[pos]+=1
    ret=[]
    for cand in cand_snp:
        pos=cand[2]
        if pos_count[pos]>=5:
            if cand not in ret:  #find the returned list of SNPs
                ret=ret+[cand]
            
    snps = ret
    output_fn = args.output_file
    zip_fn = output_fn + '.zip'
    with open(output_fn, 'w') as output_file:
        header = '>' + args.output_header + '\n>SNP\n'
        output_file.write(header)
        for x in snps:
            line = ','.join([str(u) for u in x]) + '\n'
            output_file.write(line)

        tails = ('>' + x for x in ('STR', 'CNV', 'ALU', 'INV', 'INS', 'DEL'))
        output_file.write('\n'.join(tails))

    with zipfile.ZipFile(zip_fn, 'w') as myzip:
        myzip.write(output_fn)


In [62]:
pos_count

{6990: 11,
 6589: 9,
 9180: 1,
 9175: 1,
 3608: 1,
 7750: 1,
 4543: 3,
 3135: 2,
 3114: 1,
 6408: 1,
 3451: 1,
 2044: 2,
 2026: 1,
 2021: 1,
 9211: 1,
 7670: 1,
 7408: 1,
 7410: 19,
 7412: 1,
 4793: 1,
 5828: 1,
 5807: 2,
 1752: 1,
 6784: 1,
 6775: 7,
 5148: 1,
 7292: 1,
 8777: 11,
 3916: 1,
 3907: 6,
 7079: 1,
 7071: 1,
 3244: 1,
 9309: 13,
 7928: 11,
 7944: 4,
 7945: 1,
 8288: 11,
 8300: 3,
 6665: 1,
 6677: 7,
 1538: 2,
 789: 1,
 8639: 1,
 2879: 1,
 2863: 1,
 7411: 1,
 3357: 1,
 1891: 1,
 1603: 1,
 6241: 1,
 4775: 2,
 4734: 1,
 1856: 1,
 1838: 1,
 5451: 1,
 1395: 1,
 2082: 1,
 451: 1,
 6607: 1,
 5609: 2,
 8172: 1,
 9543: 1,
 924: 1,
 393: 1,
 7207: 1,
 7191: 6,
 1034: 2,
 2772: 1,
 5630: 1,
 4138: 2,
 4139: 2,
 4140: 2,
 4142: 2,
 4143: 1,
 6301: 1,
 6282: 1,
 2498: 1,
 2489: 2,
 7840: 1,
 5081: 2,
 1486: 1,
 1503: 1,
 5460: 1,
 2178: 1,
 7032: 1,
 7025: 5,
 7016: 1,
 8656: 1,
 8658: 1,
 9645: 1,
 7363: 1,
 7346: 1,
 7340: 4,
 8165: 3,
 7487: 2,
 8694: 2,
 9719: 1,
 3754: 1,
 8624: 1

In [75]:
1.01**365

37.78343433288728

In [76]:
#chapter 9, 3.1

In [33]:
from typing import Tuple


class TrieNode(object):
    """
    Our trie node implementation. Very basic. but does the job
    """
    
    def __init__(self, char: str):
        self.char = char
        self.children = []
        # Is it the last character of the word.`
        self.word_finished = False
        # How many times this character appeared in the addition process
        self.counter = 1
    

def add(root, word: str):
    """
    Adding a word in the trie structure
    """
    node = root
    for char in word:
        found_in_child = False
        # Search for the character in the children of the present `node`
        for child in node.children:
            if child.char == char:
                # We found it, increase the counter by 1 to keep track that another
                # word has it as well
                child.counter += 1
                # And point the node to the child that contains this char
                node = child
                found_in_child = True
                break
        # We did not find it so add a new chlid
        if not found_in_child:
            new_node = TrieNode(char)
            node.children.append(new_node)
            # And then point node to the new child
            node = new_node
    # Everything finished. Mark it as the end of a word.
    node.word_finished = True


def find_prefix(root, prefix: str) -> Tuple[bool, int]:
    """
    Check and return 
      1. If the prefix exsists in any of the words we added so far
      2. If yes then how may words actually have the prefix
    """
    node = root
    # If the root node has no children, then return False.
    # Because it means we are trying to search in an empty trie
    if not root.children:
        return False, 0
    for char in prefix:
        char_not_found = True
        # Search through all the children of the present `node`
        for child in node.children:
            if child.char == char:
                # We found the char existing in the child.
                char_not_found = False
                # Assign node as the child containing the char and break
                node = child
                break
        # Return False anyway when we did not find a char.
        if char_not_found:
            return False, 0
    # Well, we are here means we have found the prefix. Return true to indicate that
    # And also the counter of the last node. This indicates how many words have this
    # prefix
    return True, node.counter

if __name__ == "__main__":
    root = TrieNode('*')
    add(root, "hackathon")
    add(root, 'hack')
    for e in root.children:
        print(e.char)
        ee=e
        while ee.children != []:
            print(ee.children[0].char)
            ee=ee.children[0]

h
a
c
k
a
t
h
o
n


In [78]:
#chapter 9, 3.2

In [116]:
myfile= open("dataset_240376_8.txt", "r")
patterns=''
count=0
pslist=[]
for line in myfile:
    if count==0:
        patterns=line.strip()
        count=1
    else:
        text=line.strip()
        temp=patterns
        pos=patterns.index(text)
        temp=patterns[pos+1:]
        pslist=pslist+[pos]
        while text in temp:
            post=temp.index(text)
            pos=pos+post+1
            pslist=pslist+[pos]
            temp=temp[post+1:]
retst=''
pslist.sort()
for ps in pslist:
    print(ps)

237
244
429
501
830
837
1137
1473
1480
1559
1676
2106
2113
2131
2310
2637
2660
2667
3094
3101
3769
3776
3804
3811
4421
4428
4500
4507
4530
4537
4629
4636
5741
5748
6169
6565
6572
6727
6734
7210
7559
7566
7951
7958


In [117]:
#chapter 9, 5.4

In [172]:
myfile= open("dataset_240378_7.txt", "r")
stno=1
for line in myfile:
    if stno==1:
        st1=line.strip()
        stno=2
    if stno==2:
        st2=line.strip()
        
suc=0
length=2
while suc==0:
    nmers_1=[]
    retlis=[]
    temp=st1
    while len(temp)>=length:
        nmers_1=nmers_1+[temp[0:length]]
        temp=temp[1:]
    for nmer in nmers_1:
        if (nmer not in st2) and (nmer not in retlis):
            retlis=retlis+[nmer]
    if len(retlis)>=1:
        suc=1
    length+=1
print(retlis)

['GAAAC', 'AACAT', 'ACATC', 'CATCT', 'ATCTT', 'TTTGG', 'TTGGA', 'GGACG', 'ACGTG', 'GTGGA', 'TGGAG', 'GGAGC', 'GCTAA', 'CTAAT', 'AATCC', 'ATCCT', 'CTTCC', 'TTCCG', 'TCCGG', 'CGGCA', 'GGCAA', 'GCAAT', 'CAATT', 'ATTAC', 'TTACG', 'CGCAG', 'GCAGC', 'CAGCC', 'GCCAC', 'CACCT', 'CTACT', 'TACTG', 'ACTGC', 'TGCGC', 'GCGCC', 'GCCGT', 'CGTCT', 'GTCTT', 'TCTTG', 'TGGAA', 'GGAAT', 'ATTAT', 'TTATC', 'CCATA', 'CATAA', 'ATAAT', 'AATCG', 'TCGAC', 'CGACT', 'GACTA', 'TAATG', 'AATGC', 'ATGCG', 'TGCGG', 'GGGGT', 'GGGTG', 'GGTGG', 'GTGGT', 'TGGTT', 'GGTTC', 'GTTCA', 'TTCAA', 'TCAAG', 'CAAGT', 'AAGTC', 'GTCAT', 'CATTA', 'TGACG', 'GACGA', 'ACGAG', 'CGAGA', 'GAGAT', 'AGATC', 'GATCC', 'TCCTC', 'TCCGC', 'CCGCT', 'CGCTT', 'GGAGG', 'GAGGA', 'AGGAG', 'AGAGC', 'GAGCG', 'AGCGG', 'GCGGC', 'GGCGA', 'GCGAT', 'GATGG', 'GGACT', 'ACTAC', 'ACCCC', 'CCCCC', 'CGCCA', 'GCCAT', 'CATAT', 'TATAT', 'TATAG', 'ATAGG', 'TAGGA', 'GGAGT', 'GAGTC', 'AGTCG', 'GTCGT', 'TCGTA', 'CGTAG', 'TAGGT', 'AGGTT', 'GGTTA', 'GTTAT', 'TATCT', 'TCTTA', 

In [160]:
#chapter 9, 5.3

In [185]:
myfile= open("dataset_240378_6.txt", "r")
stno=1
for line in myfile:
    if stno==1:
        st1=line.strip()
        stno=2
    if stno==2:
        st2=line.strip()
        
suc=0
length=20
while suc==0:
    nmers_1=[]
    retlis=[]
    temp=st1
    while len(temp)>=length:
        nmers_1=nmers_1+[temp[0:length]]
        temp=temp[1:]
    for nmer in nmers_1:
        if (nmer in st2) and (nmer not in retlis):
            retlis=retlis+[nmer]
    if len(retlis)>=1:
        suc=1
    length-=1
print(retlis)

['TATTGAGCG', 'TCACGATGC', 'TGCGTCTGG']


In [186]:
#chapter 9, 5.2

In [187]:
len('TTTCCATATACGGGACAAGGGTGAGCATTTCCGGGCTTGGATAGGGGCTGCAAGAAAATATCTGGACGTAAGAAG')

75

In [203]:
myfile= open("dataset_240378_5.txt", "r")
for line in myfile:
    txt=line.strip() 
suc=0
length=100
while suc==0:
    nmers={}
    temp=txt
    while len(temp)>=length:
        nmer=temp[0:length]
        temp=temp[1:]
        if nmer not in nmers:
            nmers[nmer]=1
        else:
            nmers[nmer]=nmers[nmer]+1
    for nmer in nmers:
        if nmers[nmer]>1:
            print(nmer)
            suc=1
    length-=1

CGGCTGGACTCCTTTGACCCGTGTAGGTGAGAATTGCCAGCGAGAAGCAGTTGTGGATGGAGCCGAAATCACAATCCAAAACCGCGTT
TCGGCTGGACTCCTTTGACCCGTGTAGGTGAGAATTGCCAGCGAGAAGCAGTTGTGGATGGAGCCGAAATCACAATCCAAAACCGCGT


In [198]:
len('CGGCTGGACTCCTTTGACCCGTGTAGGTGAGAATTGCCAGCGAGAAGCAGTTGTGGATGGAGCCGAAATCACAATCCAAAACCGCGTT')

88

In [200]:
#chapter 9, 5.1

In [20]:
class Node:
    def __init__(self, sub="", children=[]):
        self.sub = sub
        self.ch = children
 
class SuffixTree:
    def __init__(self, str):
        self.nodes = [Node()]
        for i in range(len(str)):
            self.addSuffix(str[i:])
 
    def addSuffix(self, suf):
        n = 0
        i = 0
        while i < len(suf):
            b = suf[i]
            x2 = 0
            while True:
                children = self.nodes[n].ch
                if x2 == len(children):
                    # no matching child, remainder of suf becomes new node
                    n2 = len(self.nodes)
                    self.nodes.append(Node(suf[i:], []))
                    self.nodes[n].ch.append(n2)
                    return
                n2 = children[x2]
                if self.nodes[n2].sub[0] == b:
                    break
                x2 = x2 + 1
 
            # find prefix of remaining suffix in common with child
            sub2 = self.nodes[n2].sub
            j = 0
            while j < len(sub2):
                if suf[i + j] != sub2[j]:
                    # split n2
                    n3 = n2
                    # new node for the part in common
                    n2 = len(self.nodes)
                    self.nodes.append(Node(sub2[:j], [n3]))
                    self.nodes[n3].sub = sub2[j:] # old node loses the part in common
                    self.nodes[n].ch[x2] = n2
                    break # continue down the tree
                j = j + 1
            i = i + j   # advance past part in common
            n = n2      # continue down the tree
 
    def visualize(self):
        if len(self.nodes) == 0:
            print ("<empty>")
            return
        def f(n):
            children = self.nodes[n].ch
            if len(children) == 0:
                print(self.nodes[n].sub)
                return
            print(self.nodes[n].sub)
            for c in children[:-1]:
                f(c)
            f(children[-1])
 
        f(0)
 
SuffixTree("TCAGAATCAAATTGACGGCAGGCTTCTCAACTGCAGACGCAGCCGACAACCCCTTTTCTAATGTTCCTAAAGCGTTGTTGTCACCTCACTGAGATCAGCGACAATCGTGCTGAGCTATAATGGACTCGGATATTTCCGTTCGGTGTTTGACCGGCACCAGCTGAGAAGACGCCATTCGTCGAGCGCGCCCCGCCGTGCATCGTGCCCTATACAGTGCTAAGTAGCGATCGGCATATCCTGTCTGGGGCAACCATCAATCTAGTCCACGGATGGTTCTCTCCCATGAATGACAATGTCTCGTCGTTAATAGGATGTCGCGCCCGCTCCGCAGGCAAAATACGCCATAGTAATCACGGGGCTGAATTCTGTTAGGCACACCGCTAGCGCTTTTTAGTTAAGTTTCAGGTGGTTAATGTGGAATTCCAATTGGGACCCCGCACGGTATGCAAAATTGGGCTTGACGTGGGACGTACCCGAGGAAATAACATTTGAAAGTGAGAAAAGCCTATGCATTCACGCCTCAGCCTCTTAGAGAGTGGGGTCCGTCCGGTGCTGTGAGGCGCAACAAACCATTCCCCTCATGCCATGAATTCTCAGTTGATAATCTTCATATGGACACATACTCGGCGCAGCCCCTAGTGTTTGACAGGGAGTTGTATTGTCTTGCTATCGTCATTGCGCTTCACCAAGCGATCGAATAGTCCGAGTGGTTAATGCTGACGGTAGCCTGAAGAAAATAGTTTATAGAGCAGAGGATTCACTAATGTAGAGTGGGTTCGACTAAGCTTTAGGGCCTGATGACTGCCTGCTTTGATACTAGAAACATATTCCGCTATGCTGGATGCATATTCTTTTAAACAAGAGTACTCAGGTTTACCTATGCCTAGATTGAGCTACAAAGATGGTTTACGGCCCTTAACCGAGCGCAAATTCTGGAGGAGTGCCCTCTGGGACCAACGTCGGACGCATGGAC$").visualize()


T
C
A
G
AATCAAATTGACGGCAGGCTTCTCAACTGCAGACGCAGCCGACAACCCCTTTTCTAATGTTCCTAAAGCGTTGTTGTCACCTCACTGAGATCAGCGACAATCGTGCTGAGCTATAATGGACTCGGATATTTCCGTTCGGTGTTTGACCGGCACCAGCTGAGAAGACGCCATTCGTCGAGCGCGCCCCGCCGTGCATCGTGCCCTATACAGTGCTAAGTAGCGATCGGCATATCCTGTCTGGGGCAACCATCAATCTAGTCCACGGATGGTTCTCTCCCATGAATGACAATGTCTCGTCGTTAATAGGATGTCGCGCCCGCTCCGCAGGCAAAATACGCCATAGTAATCACGGGGCTGAATTCTGTTAGGCACACCGCTAGCGCTTTTTAGTTAAGTTTCAGGTGGTTAATGTGGAATTCCAATTGGGACCCCGCACGGTATGCAAAATTGGGCTTGACGTGGGACGTACCCGAGGAAATAACATTTGAAAGTGAGAAAAGCCTATGCATTCACGCCTCAGCCTCTTAGAGAGTGGGGTCCGTCCGGTGCTGTGAGGCGCAACAAACCATTCCCCTCATGCCATGAATTCTCAGTTGATAATCTTCATATGGACACATACTCGGCGCAGCCCCTAGTGTTTGACAGGGAGTTGTATTGTCTTGCTATCGTCATTGCGCTTCACCAAGCGATCGAATAGTCCGAGTGGTTAATGCTGACGGTAGCCTGAAGAAAATAGTTTATAGAGCAGAGGATTCACTAATGTAGAGTGGGTTCGACTAAGCTTTAGGGCCTGATGACTGCCTGCTTTGATACTAGAAACATATTCCGCTATGCTGGATGCATATTCTTTTAAACAAGAGTACTCAGGTTTACCTATGCCTAGATTGAGCTACAAAGATGGTTTACGGCCCTTAACCGAGCGCAAATTCTGGAGGAGTGCCCTCTGGGACCAACGTCGGACGCATGGAC$
C
GACAATCGTGCTGAGCTA

In [ ]:
A
T
AAATG$
G$
A
ATG$
TG$
T
AAATG$
G$
G$
$

In [201]:
#chapter 9, 6.1

In [214]:
import copy
myfile= open("dataset_240379_2.txt", "r")
for line in myfile:
    txt=line.strip() 
sublis=[]
while len(txt)>0:
    sublis=sublis+[txt]
    txt=txt[1:]
sublis_s=copy.copy(sublis)
sublis_s.sort()
for i in range(len(sublis_s)):
    sublis_s[i]=sublis.index(sublis_s[i])
ret=''
for sub in sublis_s:
    ret=ret+str(sub)+', '
print(ret)

901, 264, 371, 743, 411, 3, 265, 372, 696, 78, 744, 444, 412, 27, 219, 708, 4, 266, 688, 109, 650, 295, 433, 373, 113, 312, 697, 15, 711, 604, 48, 79, 745, 449, 763, 445, 96, 292, 760, 458, 778, 118, 869, 303, 536, 841, 143, 754, 202, 281, 7, 594, 413, 28, 220, 682, 655, 552, 709, 116, 5, 37, 484, 882, 544, 461, 235, 532, 267, 689, 355, 110, 651, 296, 849, 39, 749, 519, 802, 287, 396, 863, 434, 618, 69, 468, 674, 315, 642, 740, 441, 309, 374, 173, 722, 377, 206, 135, 500, 388, 568, 486, 891, 798, 271, 583, 884, 262, 76, 686, 114, 233, 847, 466, 313, 720, 386, 231, 718, 546, 874, 858, 638, 405, 664, 188, 698, 176, 669, 541, 16, 712, 276, 152, 472, 605, 49, 80, 193, 630, 746, 548, 450, 165, 19, 764, 360, 876, 725, 781, 65, 522, 834, 327, 733, 805, 560, 380, 121, 446, 860, 259, 463, 817, 576, 97, 237, 369, 409, 293, 761, 290, 534, 459, 285, 640, 889, 269, 872, 779, 119, 407, 870, 304, 347, 666, 190, 537, 842, 700, 178, 144, 626, 829, 755, 337, 227, 157, 691, 400, 93, 671, 203, 557, 282, 8

In [216]:
#chapter 9, 7.1

In [226]:
myfile= open("dataset_240380_5.txt", "r")
for line in myfile:
    txt=line.strip() 
sublis=[]
for i in range(len(txt)):
    sub=txt[i:]+txt[:i]
    sublis=sublis+[sub]
sublis.sort()
ret=''
for sub in sublis:
    ret=ret+sub[-1]
print(ret)

CCTGCTAGGCCTTCGATGCAGCCTTACTGCTACGCGGCCCAAGTGAGTATCTACTAATGCAGTAGAGCGGGCAGCGCGCGGAAGCGTGATTCGAAGTTAATTGCATCTGGACCGGATTGTAAACAACGGTTCATAAAGTTGAGGATGTATTGAAGATAACGAGGAATGCTGTTGATGGCATTAGACGACTGATGGTTGTGCGCTCCGAATCCACCGAACATG$GTGGCCCTCTACAGGCCGATTCGTGCACAGTATGTTAGATTGACGCGCCTACTAGTTGAGACCTAACGGAGACCGCCCCTCCGGATACTCAACGGGTGCACTCTCCTGCGAGAGGACAAATATCTACGGCCTGGGTGGGCAGGGGGGCTAAGTGCTACTCGAGGAGGATCAGGGTAAAATAGGGTGCTAATCAATGGAACGCTGGTCCCCGGACGACCGCCTTTGAGCCAGTTCAAATGCATTATCAGACTTGTCGGCAGAGAGTCCGCCCTGTCTGGCCAATACTGCTGCTGAACAAGAATAATGTCGGCGATATGAAACAGTCTGGTTGCTAATGGACCTGTAGAGACGGAAAGAATGGAATCTCTCATACGATTTTCTTCTGCCTTCAATAGTGAGATGCGCCCATGCAATGGCTTCCTCTCACGTTACTTCAACCGTCTAGGAAGCAATTAACCGCGGAATATAGGTTCGCGCCCTTTAGAGGAGTGTATCAAGGCTCGATCAGATTCGTGGGGATTGCAAGGATGTTGCAGCTCACGTGTAAGCGTAGCCTCATACTGGCATAGCGTCTCAATAAAAGCTCTCTTTAGTAGTTGTTTTCTGAGGGACCGAGCCCTTCCCTTCACACTCCTGCTTGCGTAATAATAGCTGACGGACATAGCCCGGCAGGGCTATAGCATACAAGGTGATGGATTCTGCAATTACCGGCCATTTACAATGGATCTTATATTTGTA


In [224]:
ret.index('$')

222

In [225]:
#chapter 9, 9.1

In [259]:
import copy
myfile= open("dataset_240382_10.txt", "r")
for line in myfile:
    txt=line.strip() 
bwt=[]
for i in txt:
    bwt=bwt+[i]
fc=copy.copy(bwt)
fc.sort()
ret='$'
ap=1
while len(ret) < len(txt):
    char=ret[-1]
    bwt_temp=copy.copy(bwt)
    ind=0
    while ap>1:
        ind_t=bwt_temp.index(char)
        ind=ind+ind_t+1
        bwt_temp=bwt_temp[ind_t+1:]
        ap=ap-1
    ind=ind+bwt_temp.index(char)
    ret=ret+fc[ind]
    ap=fc[:ind+1].count(fc[ind])
print(ret)

$GGTTCACAAGCGAGTTCATATACTAATACCTCTTCCTGACTCCATCAACT


In [ ]:
#chapter 9, 10.1

In [11]:
import copy
myfile= open("dataset_240383_8.txt", "r")
c=0
for line in myfile:
    if c==0:
        txt=line.strip()
        c=1
    if c==1:
        patterns=line.strip().split(' ')
bwt=[]
for i in txt:
    bwt=bwt+[i]
fc=copy.copy(bwt)
fc.sort()
ret='$'
ap=1
while len(ret) < len(txt):
    char=ret[-1]
    bwt_temp=copy.copy(bwt)
    ind=0
    while ap>1:
        ind_t=bwt_temp.index(char)
        ind=ind+ind_t+1
        bwt_temp=bwt_temp[ind_t+1:]
        ap=ap-1
    ind=ind+bwt_temp.index(char)
    ret=ret+fc[ind]
    ap=fc[:ind+1].count(fc[ind])
ret_lis=[]
for p in patterns:
    ret_lis=ret_lis+[ret.count(p)]
nstr=''
for n in ret_lis:
    nstr=nstr+str(n)+' '
fc.index('T')

36

In [270]:
#chapter 9, 11.1

In [18]:
import copy
myfile= open("dataset_240384_7.txt", "r")
c=0
for line in myfile:
    if c==0:
        txt=line.strip()
        c=1
    if c==1:
        patterns=line.strip().split(' ')
bwt=[]
for i in txt:
    bwt=bwt+[i]
fc=copy.copy(bwt)
fc.sort()
#till here
firstDict={}
symb=fc[0]
for i in range (len(fc)):
    if fc[i]!=symb or (fc[i] not in firstDict):
        symb=fc[i]
        firstDict[symb]=i

def betterBWMatching(FirstOccurrence, LastColumn, Pattern):
    top=0
    bottom=len(LastColumn)-1
    while top<=bottom:
        if len(Pattern)>0:
            symbol=Pattern[-1]
            Pattern=Pattern[:-1]
            if symbol in LastColumn[top:bottom+1]:
                top=FirstOccurrence[symbol]+LastColumn[:top].count(symbol)
                bottom=FirstOccurrence[symbol]+LastColumn[:bottom+1].count(symbol)-1
            else:
                return 0
        else:
            return bottom-top+1

ret=''
for p in patterns:
    ret=ret+str(betterBWMatching(firstDict,txt,p))+' '

In [271]:
#chapter 9, 13.1

In [274]:
myfile= open("dataset_240386_4.txt", "r")
patterns=''
count=0
pslist=[]
for line in myfile:
    if count==0:
        patterns=line.strip()
        count=1
    else:
        text=line.strip()
        temp=patterns
        if text in temp:
            pos=patterns.index(text)
            temp=patterns[pos+1:]
            pslist=pslist+[pos]
        while text in temp:
            post=temp.index(text)
            pos=pos+post+1
            pslist=pslist+[pos]
            temp=temp[post+1:]
retst=''
pslist.sort()
for ps in pslist:
    print(ps)

2
12
13
14
15
24
25
26
27
28
29
31
39
40
41
42
43
44
54
63
64
65
66
67
68
69
70
84
87
88
89
91
100
103
123
124
125
126
127
128
130
140
141
148
149
150
151
160
168
169
185
193
194
195
196
197
198
199
200
201
202
210
211
212
213
214
234
235
236
237
239
240
241
242
254
264
265
266
267
277
278
291
292
293
294
295
296
310
319
320
331
338
343
347
360
372
395
402
403
421
424
425
426
427
428
429
430
431
432
433
439
440
441
443
447
448
449
450
451
457
462
485
497
500
507
508
509
510
511
512
513
514
522
528
539
550
553
559
568
569
570
571
572
573
577
582
583
584
585
586
587
588
589
602
607
612
615
627
631
638
639
640
648
649
650
651
653
654
655
674
675
685
695
696
704
705
713
723
724
725
726
727
728
729
730
731
732
748
754
755
756
757
758
759
760
761
773
784
788
791
794
797
807
816
817
818
819
820
821
822
829
830
831
832
833
834
835
836
837
855
856
857
865
866
867
868
869
876
877
878
894
895
896
897
898
899
900
907
908
909
910
911
912
913
914
915
924
925
926
930
931
932
939
961
971
972
973
974
9

In [275]:
#chapter 9, 14.1

In [65]:
import copy
myfile= open("dataset_240387_10.txt", "r")
c=0
for line in myfile:
    if c==0:
        txt=line.strip()
        c=1
    elif c==1:
        patterns=line.strip().split(' ')
        lens=[]
        for p in patterns:
            if len(p) not in lens:
                lens.append(len(p))
        c=2
    elif c==2:
        thres=line.strip()
ht_list=[]
for num in lens:
    hash_table={}
    for i in range(len(txt)-num+1):
        seq=txt[i:i+num]
        if seq not in hash_table:
            hash_table[seq]=[i]
        else:
            hash_table[seq]=hash_table[seq]+[i]
    ht_list.append(hash_table)
ret=[]
for p in patterns:
    length=len(p)
    ht=ht_list[lens.index(length)]
    for kmer in ht:
        if compare_two_seq(p,kmer,int(thres)):
            ret=ret+ht[kmer]
ret.sort()
ret

[4,
 7,
 25,
 27,
 30,
 30,
 33,
 34,
 43,
 52,
 58,
 60,
 74,
 77,
 81,
 81,
 93,
 116,
 116,
 120,
 124,
 131,
 136,
 136,
 136,
 138,
 138,
 146,
 167,
 168,
 169,
 177,
 185,
 193,
 215,
 221,
 224,
 227,
 234,
 237,
 246,
 248,
 249,
 249,
 253,
 257,
 259,
 260,
 270,
 278,
 280,
 281,
 283,
 287,
 293,
 302,
 305,
 307,
 314,
 318,
 324,
 325,
 326,
 328,
 334,
 337,
 361,
 362,
 365,
 369,
 373,
 378,
 384,
 395,
 397,
 407,
 408,
 412,
 413,
 414,
 415,
 424,
 426,
 432,
 438,
 440,
 446,
 447,
 448,
 460,
 466,
 472,
 474,
 476,
 479,
 480,
 483,
 485,
 487,
 488,
 508,
 510,
 516,
 522,
 523,
 528,
 529,
 535,
 536,
 544,
 545,
 545,
 550,
 557,
 564,
 569,
 571,
 582,
 608,
 610,
 610,
 612,
 613,
 615,
 619,
 620,
 620,
 637,
 640,
 641,
 647,
 649,
 663,
 667,
 673,
 681,
 688,
 692,
 694,
 695,
 698,
 700,
 702,
 703,
 705,
 708,
 708,
 714,
 716,
 719,
 724,
 733,
 734,
 741,
 741,
 747,
 747,
 756,
 766,
 781,
 781,
 793,
 802,
 803,
 806,
 809,
 828,
 830,
 833,
 849,

In [59]:
def compare_two_seq(seq,kmer,thres):
    count=0
    for i in range(len(seq)):
        if seq[i]!=kmer[i]:
            count+=1
        if count>thres:
            return False
    return True

In [69]:
compare_two_seq('ATT','ACA',1)

False